In [1]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import json

In [2]:
event_cutoff = 300
small_pull_cutoff = 0.2
raw_data_file = "./data/raw/SheetPullData.csv"
processed_data_file = "./data/processed_sheets_pull.csv"

In [3]:
# helper functions
def clip(x):
    if x >= 0: return x
    return 0

def process_timestamp(timestamp_str):
    # TODO: pad timeframe into standard UNIX time?
    return timestamp_str

def enforce_list_identical(some_list):
    assert len(some_list) > 0
    elem = some_list[0]
    for obj in some_list:
        assert elem == obj, "Mismatch! First element: {} Current element: {}".format(elem, obj)

In [4]:
# Step 1: read dataset while merging events
def prep_feature(event_list_rows):
    '''
    event: a list of rows where each element is a dict with column
            names in SheetPullData.csv as keys. List may or may
            not contain multiple rows.
    
    Return: a pandas.DataFrame object that can be appended
    '''
    
    # Read out all columns in raw dataset
    HH_ID_list = [d['HH_ID'] for d in event_list_rows]
    roll_ID_list = [d['Roll_ID'] for d in event_list_rows]
    roll_Type_list = [d['Roll_Type'] for d in event_list_rows]
    timestamp_list = [process_timestamp(d['Timestamp']) for d in event_list_rows]
    seconds_since_last_list = [float(d['Time_since_last_pull']) for d in event_list_rows]
    seconds_duration = []
    if(len(seconds_since_last_list) > 0): seconds_duration = [seconds_since_last_list[i] for i in range(1,len(seconds_since_last_list))]
    sheets_cnt_list = [float(d['Sheets']) for d in event_list_rows]
    
    # Sanity Check
    enforce_list_identical(HH_ID_list)
    enforce_list_identical(roll_ID_list)
    enforce_list_identical(roll_Type_list)
    assert len(HH_ID_list) == len(timestamp_list)
    
    # Preprocess rule 1: clip negative values
    sheets_cnt_list = [clip(i) for i in sheets_cnt_list]
    
    # build features
    cur_row = {
        'seconds_since_previous_pull': [seconds_since_last_list[0]],
        'HH_ID': [HH_ID_list[0]],
        'roll_ID': [roll_ID_list[0]],
        'roll_type': [roll_Type_list[0]],
        'multiple_pull_flag': [(len(HH_ID_list) > 1)],
        'starting_time': [timestamp_list[0]],
        'number_pulled': [len(HH_ID_list)],
        'total_sheets_used': [sum(sheets_cnt_list)],
        'average_sheets_between_pulls':[np.mean(sheets_cnt_list[1:]) if (len(HH_ID_list) > 1) else 0],
        'max_sheets_pull' :[max(sheets_cnt_list)],
        'max_min_pull_diff': [max(sheets_cnt_list) - min(sheets_cnt_list)],
        'elapsed_time': [sum(seconds_since_last_list[1:])],
        'average_time_between_pulls': [np.mean(seconds_since_last_list[1:]) if (len(HH_ID_list) > 1) else 0],
        'max_time_duration': [max(seconds_duration) if len(seconds_duration) > 0 else 0],
    }
    return pd.DataFrame(data = cur_row)

In [5]:
# remove cached variable
try:
    delete(ret_df)
except NameError:
    pass

with open(raw_data_file, newline = '') as csvfile:
    reader = csv.DictReader(csvfile)
    # read to memory
    all_rows = list(reader)
    first_row = all_rows[0]
    event_rows = [first_row]
    for row in tqdm(all_rows[1:]):
        if float(row['Time_since_last_pull']) < event_cutoff:
            event_rows.append(row)
        else:
            new_df = prep_feature(event_rows)
            try:
                ret_df = ret_df.append(new_df)
            except NameError:
                ret_df = new_df
            event_rows = [row]
    if event_rows:
        new_df = prep_feature(event_rows)
        ret_df = ret_df.append(new_df)

100%|██████████| 12784/12784 [00:11<00:00, 1159.56it/s]


In [6]:
ret_df = ret_df[ret_df['total_sheets_used'] > 1]

In [7]:
ret_df

,seconds_since_previous_pull,HH_ID,roll_ID,roll_type,multiple_pull_flag,starting_time,number_pulled,total_sheets_used,average_sheets_between_pulls,max_sheets_pull,max_min_pull_diff,elapsed_time,average_time_between_pulls,max_time_duration
0,4282.0,A,1200,Small,False,2019-10-29 05:51:42,1,3.991088,0.000000,3.991088,0.000000,0.0,0.00,0.0
0,2394.0,A,1200,Small,False,2019-10-29 07:13:21,1,4.277735,0.000000,4.277735,0.000000,0.0,0.00,0.0
0,14200.0,A,1200,Small,False,2019-10-29 11:10:01,1,2.363388,0.000000,2.363388,0.000000,0.0,0.00,0.0
0,828.0,A,1200,Small,True,2019-10-29 11:23:49,2,2.980160,2.665750,2.665750,2.351341,4.0,4.00,4.0
0,12312.0,A,1200,Small,False,2019-10-29 14:49:05,1,2.186278,0.000000,2.186278,0.000000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1919.0,D,13655,Big,True,2020-01-13 07:05:46,3,16.694831,5.692457,6.875632,2.366350,32.0,16.00,26.0
0,347.0,D,13655,Big,False,2020-01-13 07:12:05,1,1.861284,0.000000,1.861284,0.000000,0.0,0.00,0.0
0,29363.0,D,13655,Big,True,2020-01-13 15:21:28,2,9.511939,3.260943,6.250996,2.990052,6.0,6.00,6.0
0,4364.0,D,13655,Big,True,2020-01-13 16:34:18,5,29.036462,6.066759,11.417136,11.062394,61.0,15.25,42.0


In [8]:
ret_df.to_csv('./data/processed_sheets_pull.csv')